In [1]:
import os, sys

In [2]:
%pwd

u'/home/ubuntu/courses/my_nbs'

In [3]:
current_dir = os.getcwd()

In [5]:
LESSON_HOME_DIR = current_dir

In [6]:
DATA_HOME_DIR = current_dir+'/data/kaggle_dogs_cats'

In [7]:
sys.path[0]

''

In [8]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [9]:
from utils import *
from vgg16 import Vgg16

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [10]:
%matplotlib inline

## Action Plan
1. Create Validation and Sample sets
2. Rearrange image files into their respective directories 
3. Finetune and Train model
4. Generate predictions
5. Validate predictions
6. Submit predictions to Kaggle

In [12]:
#Create directories
%cd $DATA_HOME_DIR
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

/home/ubuntu/courses/my_nbs/data/kaggle_dogs_cats
mkdir: cannot create directory ‘results’: File exists


In [13]:
%cd $DATA_HOME_DIR/train

/home/ubuntu/courses/my_nbs/data/kaggle_dogs_cats/train


In [14]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): os.rename(shuf[i], DATA_HOME_DIR+'/valid/' + shuf[i])

In [15]:
from shutil import copyfile

In [16]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(200): copyfile(shuf[i], DATA_HOME_DIR+'/sample/train/' + shuf[i])

In [17]:
%cd $DATA_HOME_DIR/valid

/home/ubuntu/courses/my_nbs/data/kaggle_dogs_cats/valid


In [18]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(50): copyfile(shuf[i], DATA_HOME_DIR+'/sample/valid/' + shuf[i])

### Rearrange image files into their respective directories

In [19]:
#Divide cat/dog images into separate directories

%cd $DATA_HOME_DIR/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

/home/ubuntu/courses/my_nbs/data/kaggle_dogs_cats/sample/train
/home/ubuntu/courses/my_nbs/data/kaggle_dogs_cats/sample/valid
/home/ubuntu/courses/my_nbs/data/kaggle_dogs_cats/valid
/home/ubuntu/courses/my_nbs/data/kaggle_dogs_cats/train


In [20]:
# Create single 'unknown' class for test set
%cd $DATA_HOME_DIR/test
%mv *.jpg unknown/

/home/ubuntu/courses/my_nbs/data/kaggle_dogs_cats/test


## Finetuning and Training

In [21]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' #'/sample/'
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

/home/ubuntu/courses/my_nbs/data/kaggle_dogs_cats


In [22]:
#import Vgg16 helper class
vgg = Vgg16()

Weights loaded
Getting Models
models done


In [23]:
#Set constants. You can experiment with no_of_epochs to improve the model
batch_size=64
no_of_epochs=3

In [24]:
#Finetune the model
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)
vgg.finetune(batches)

#Not sure if we set this for all fits
vgg.model.optimizer.lr = 0.01

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
#Notice we are passing in the validation dataset to the fit() method
#For each epoch we test our model against the validation set
latest_weights_filename = None
for epoch in range(no_of_epochs):
    print "Running epoch: %d" % epoch
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch
    vgg.model.save_weights(results_path+latest_weights_filename)
print "Completed %s fit operations" % no_of_epochs

Running epoch: 0
Starting to fit data
Epoch 1/1
 8832/23000 [==========>...................] - ETA: 450s - loss: 0.4602 - acc: 0.9578